Virginia updates at noon EDT daily.

In [8]:
from selenium import webdriver
import time
import pandas as pd
import pendulum
import re
import cv2
import yaml
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=[Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0]")

In [9]:
from PIL import Image
from pytesseract import image_to_string, image_to_data, image_to_pdf_or_hocr
import base64

In [10]:
def save_tb_image(driver, canvas, filename):
    """get the canvas as a PNG base64 string"""
    canvas_base64 = driver.execute_script("return arguments[0].toDataURL('image/png').substring(21);", canvas)

    # decode
    canvas_png = base64.b64decode(canvas_base64)

    # save to a file
    with open(f"{filename}_{scrape_timestamp}.png", 'wb') as f:
        f.write(canvas_png)

In [11]:
snip = """/* global arguments */
(function (element, deltaY, offsetX, offsetY) {
    var box = element.getBoundingClientRect();
    var clientX = box.left + (offsetX || box.width / 2);
    var clientY = box.top + (offsetY || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
        if (e === element) {
            target.dispatchEvent(new MouseEvent("mouseover", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY
            }));
            target.dispatchEvent(new MouseEvent("mousemove", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY
            }));
            target.dispatchEvent(new WheelEvent("wheel", {
                view: window,
                bubbles: true,
                cancelable: true,
                clientX: clientX,
                clientY: clientY,
                deltaY: deltaY
            }));
            return "";
        }
    }
    return "Element is not interactable";
}).apply(null, arguments);
"""

In [12]:
def simulate_wheel(element, deltaY=120, offsetX=0, offsetY=0):
    """Helper to simulate a mouse wheel scroll."""
    error = element._parent.execute_script(snip, element, deltaY, offsetX, offsetY)
    if error:
        raise WebDriverException(error)

In [13]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())

In [14]:
state = 'VA'

In [15]:
scrape_timestamp = pendulum.now().strftime('%Y%m%d%H%M%S')

In [16]:
# url = 'http://www.vdh.virginia.gov/coronavirus/'
# url = 'https://public.tableau.com/views/VirginiaCOVID-19Dashboard/VirginiaCOVID-19Dashboard?%3Aembed=y&%3AshowVizHome=no&%3Adisplay_count=y&%3Adisplay_static_image=y&%3AbootstrapWhenNotified=true'
url = 'https://public.tableau.com/views/VirginiaCOVID-19Dashboard/VirginiaCOVID-19Dashboard?%3Aembed=y&%3AshowVizHome=no&%3Adisplay_count=y&%3Adisplay_static_image=y&%3Adevice=desktop'

In [40]:
def fetch():
    driver = webdriver.Chrome('../20190611 - Parts recommendation/chromedriver', options=chrome_options)

    driver.get(url)

    # Virginia uses Tableau iframe for data
    tbl = driver.find_element_by_id("main-content")
    time.sleep(5)

    # grab the table with county level cases
    datatbl = tbl.find_element_by_id('tabZoneId115')

    # scroll data table into view
    driver.execute_script("arguments[0].scrollIntoView(true);", datatbl)

    # Virginia uses a public Tableau dashboard to report total cases. Need to grab the data table's canvas element and OCR.

    # control how many mouse wheel scrolls to perform.
    scroll_cnt = 4

    for i in range(scroll_cnt):
        # grab all canvas elements
        canvases = datatbl.find_elements_by_css_selector('canvas')

        # save image
        for idx, canvas in enumerate(canvases):
            save_tb_image(driver, canvas, f"{config['data_source_backup_folder']}/{state}_county_{i}_{idx}")

        # scroll canvas down
        simulate_wheel(canvases[1], 500)
        time.sleep(3)

    driver.close()
        
    # Join images together so data table OCR works properly
    for i in range(scroll_cnt):
        im1 = cv2.imread(f"{config['data_source_backup_folder']}/{state}_county_{i}_1_{scrape_timestamp}.png")
        im2 = cv2.imread(f"{config['data_source_backup_folder']}/{state}_county_{i}_2_{scrape_timestamp}.png")
        im_h = cv2.hconcat([im1[:, 300:,:], im2[:,400:600,:]]) # trimming down the width of the positive cases so teseract works better
        cv2.imwrite(f"{config['data_source_backup_folder']}/{state}_county_{i}_{scrape_timestamp}_final.png", im_h)

    # OCR combined image. 
    data = []
    for i in range(scroll_cnt):
        tmp = image_to_string(f"{config['data_source_backup_folder']}/{state}_county_{i}_{scrape_timestamp}_final.png", config='--psm 6').split('\n')
        data += [re.search('(.*) (\d*)$', row).groups() for row in tmp]
        
    return pd.DataFrame(set(data), columns=['county', 'positive_cases'])

In [1]:
def save(df):
    df.to_csv(f"{config['data_folder']}/{state}_county_{scrape_timestamp}.txt", sep='|', index=False)

    # no need to backup virginia page
#     with open(f"{config['data_source_backup_folder']}/{state}_county_{scrape_timestamp}.html", 'w') as f:
#         f.write(driver.page_source)

In [ ]:
def run():
    df = fetch()
    save(df)